In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import libcst as cst

from spot.tokenized_src import TokenizedSrc
from spot.utils import Path

ex_code='''# document comment 1
  # document comment 2
"""String document commnet"""
import os; import spot;
from sys import argv, exit
# after import
def catch_permission_denied(function):
    import some.inner.imports
    """
    Decorator to catch :class:`psycopg2.ProgrammingError` exceptions with the
    ``INSUFFICIENT_PRIVILEGE`` error code and rethrow them as
    :class:`~werkzeug.exceptions.Forbidden` exceptions instead.
    """
    @wraps(function)
    def decorated(x: str, y: int) -> str:
        try:
            # comment 1
            # comment 1 cont
            return function(*args, **kwargs)

        except InsufficientPrivilege as error:
            LOG.error("Forbidden: %s", error) # comment 2
            raise Forbidden()

    return decorated
'''
print(TokenizedSrc.parse(ex_code, Path("test_file"), Path("test_repo")))

========TokenizedSrc========
file:test_file
repo:test_repo
--------Preamble--------
import os; 
from sys import argv, exit
import spot;

--------Main Code--------
def catch_permission_denied(function):
    import some.inner.imports
    @wraps(function)
    def decorated(x: str, y: int) -> str:
        try:
            return function(*args, **kwargs)

        except InsufficientPrivilege as error:
            LOG.error("Forbidden: %s", error) 
            raise Forbidden()

    return decorated

========End of TokenizedSrc========


In [23]:
from spot.data import SrcDataset
from spot.utils import DefaultTokenizer, proj_root

simple_dataset = SrcDataset.from_repos(
    proj_root() / "data",
    [proj_root() / "data/code"],
    DefaultTokenizer,
    drop_comments=True,
    max_workers=10,
    label_ratio=0.5,
)

n_files_too_wide: 0
too_wide_ratio: 0
drop_comments: True


mask_type_annots:   0%|          | 0/9 [00:00<?, ?it/s]

dict_to_tokenized_src:   0%|          | 0/9 [00:00<?, ?it/s]

In [25]:
file2preds = {
    (proj_root() / "data/code/bad_code_1.py"): {
        1: "str",
        2: "str",
    },
    (proj_root() / "data/code/bad_code_2.py"): {
        0: "int",
    },
    (proj_root() / "data/code/dummy/dummy_2.py"): {
        0: "str",
    }
}
fdbks = simple_dataset._get_type_checker_feedback_iso(
    file2preds,
    max_workers=20,
)
for f in fdbks:
    f.pretty_print()

type_check_src:   0%|          | 0/3 [00:00<?, ?it/s]

Feedbacks:
MypyFeedback(position=CodePosition(line=14, column=12), message='Incompatible return value type (got "str", expected "int") ', error_code='return-value')
======= New code =======
 1|  from typing import Any # SPOT
 2|  from typing import Any
 3|  
 4|  def fib(n: str) -> Any:
 5|      if n == 0:
 6|          return 0
 7|      elif n == 1:
 8|          return 1
 9|      else:
10|          return fib(n-1) + fib(n-2)
11|  
12|  def t_add(x: str, y: str) -> int:
13|      r = x + y
14|      return r
15|  
16|  x: int = fib(3)
17|  bad_y: str = 1
Feedbacks:
MypyFeedback(position=CodePosition(line=5, column=5), message='Argument 1 to "fib" has incompatible type "int"; expected "str" ', error_code='arg-type')
======= New code =======
1|  from typing import Any # SPOT
2|  from bad_code_1 import fib
3|  
4|  i: int = 4
5|  fib(i)
6|  
Feedbacks:
MypyFeedback(position=CodePosition(line=4, column=10), message='Incompatible types in assignment (expression has type "int", variable has typ

In [4]:
from spot.data import Path, type_check_src, type_check_src_in_project

src_to_check = simple_dataset.get_src_by_file(Path("bad_code_2.py"))
type_check_src(src_to_check, {0: "int"}).pretty_print()


Feedbacks:
======= New code =======
from typing import Any # SPOT
from bad_code_1 import fib

i: int = 4
fib(i)



In [7]:
from spot.data import type_check_src, type_check_src_in_project
import shutil

src_to_check = simple_dataset.get_src_by_file(Path("bad_code_2.py"))
temp_dir = (proj_root() / "mypy_temp/test_dir")
shutil.rmtree(temp_dir, ignore_errors=True)

type_check_src_in_project(
    src_to_check,
    {0: "int"},
    project_files=(proj_root() / "data/code").glob("**/*.py"),
    project_root=(proj_root() / "data/code"),
    temp_dir=temp_dir,
).pretty_print()


Feedbacks:
MypyFeedback(position=CodePosition(line=5, column=5), message='Argument 1 to "fib" has incompatible type "int"; expected "str" ', error_code='arg-type')
======= New code =======
from typing import Any # SPOT
from bad_code_1 import fib

i: int = 4
fib(i)



In [31]:
print(cst.parse_module((proj_root() / "src/spot/data.py").read_text()).visit(CommentRemover()).code)

import logging
import pickle
import random
import shutil
import subprocess
import warnings
from copy import copy
from dataclasses import dataclass
from datetime import datetime
from typing import *

import dateparser
from datasets import Dataset

from spot.type_env import (
    AnnotCat,
    AnnotInfo,
    AnnotPath,
    MypyChecker,
    PythonType,
    apply_annotations,
    collect_annots_info,
    collect_user_annotations,
    normalize_type,
    parse_type_expr,
    parse_type_from_ast,
)
from spot.utils import *

warnings.filterwarnings(
    "ignore",
    message="The localize method is no longer necessary, as this time zone supports the fold attribute",
)


@dataclass
class GitRepo:
    author: str
    name: str
    url: str
    stars: int
    forks: int
    lines_of_code: Optional[int] = None
    last_update: Optional[datetime] = None
    n_type_annots: Optional[int] = None
    n_type_places: Optional[int] = None

    def authorname(self):
        return self.author + "__" + sel

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from typing import *

import pandas as pd
import plotly.express as px

from spot.data import GitRepo, ModuleRemapUnpickler
from spot.type_env import (
    AnnotPath,
    MypyChecker,
    SelectAnnotations,
    TypeInfAction,
    TypeInfEnv,
    TypeInfState,
    collect_annotations,
    mypy_checker,
)
from spot.utils import cst, proj_root, read_file, seq_flatten, tqdm, write_file

os.chdir(proj_root())

datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"

useful_repos_path = proj_root() / "scripts" / "useful_repos.pkl"
rename_module = lambda n: "spot.data" if n == "spot.data_prepare" else n
with useful_repos_path.open("rb") as f:
    useful_repos: list[GitRepo] = ModuleRemapUnpickler(f, rename_module).load()

In [2]:
# loading pre-trained model and tokenizer

model_dir = datadir / "checkpoints/saved/SPOT-CodeT5-with_margin/"
# model_dir = datadir / "checkpoints/saved/SPOT-CodeT5-no_margin/"

import torch
from transformers import (
    DataCollatorForSeq2Seq,
    RobertaTokenizer,
    T5ForConditionalGeneration,
)
from transformers.models.t5 import T5ForConditionalGeneration

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer: RobertaTokenizer = RobertaTokenizer.from_pretrained(model_dir)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained(
    model_dir
).to(device)
max_target_length = 128


/home/jiayi/Projects/SPOT/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [3]:
import shutil

inference_dir = Path("data/code_output/inference")
if inference_dir.exists():
    shutil.rmtree(inference_dir)
inference_dir.mkdir(parents=True)
write_file(inference_dir / "env_code_1.py", read_file("data/code/env_code_1.py"))
write_file(inference_dir / "env_code_2.py", read_file("data/code/env_code_2.py"))


In [9]:
from spot.data import mask_type_annots, output_ids_as_types, tokenize_masked

test_code = """
@dataclass
class GitRepo:
    author: str
    name: str
    url: str
    stars: int
    forks: int

    def authorname(self):
        return self.author + "__" + self.name

    def repo_dir(self, repos_dir: Path) -> Path:
        return repos_dir / "downloaded" / self.authorname()

    def download(self, repos_dir: Path, timeout=None) -> bool:
        pass
"""


def run_model(code: str, num_beams=16):
    masked = mask_type_annots((Path("no_source"), code))
    tks = tokenize_masked(masked, tokenizer, device)
    input_ids = tks["input_ids"]
    with torch.no_grad():
        loss = model.forward(**tks).loss
        dec = model.generate(
            input_ids,
            max_length=max_target_length,
            num_beams=num_beams,
            # do_sample=True,
        )[0]
    return {
        "loss": loss,
        "predicted_types": output_ids_as_types(dec, tokenizer),
        "labels": output_ids_as_types(tks["labels"][0], tokenizer),
        "generation": tokenizer.decode(dec),
        "input_ids": input_ids[0],
        "output_ids": dec,
        "annots_info": masked["annots_info"],
    }


result = run_model(test_code, num_beams=10)
result["loss"]


tensor(2.9077, device='cuda:0')

In [24]:
from spot import PythonType
from spot.type_env import apply_annotations


def type_to_annot(ty: PythonType) -> str:
    return cst.Annotation(cst.parse_expression(str(ty)))


def run_aug_model(src: Path, cwd: Path):
    result = run_model(read_file(src), num_beams=10)
    pred_annots = {
        info.path: type_to_annot(t)
        for info, t in zip(result["annots_info"], result["predicted_types"])
    }
    m1 = apply_annotations(cst.parse_module(read_file(src)), pred_annots)
    write_file(src, m1.code)
    checker_r = MypyChecker.check_project(src, cwd)
    pos_to_preds = {
        info.annot_range: str(ty)
        for info, ty in zip(result["annots_info"], result["predicted_types"])
    }
    return {
        "model_result": result,
        "module": m1,
        "checker_feedback": checker_r,
        "pos_to_preds": pos_to_preds,
    }


aug_r = run_aug_model(inference_dir / "env_code_2.py", inference_dir)


In [33]:
from spot.utils import patch_code_with_extra
print("---- predicted types ----")
print(aug_r['model_result']['predicted_types'])
print("---- model output ----")
print(tokenizer.decode(aug_r["model_result"]["output_ids"], skip_special_tokens=False))
print("---- checker_feedback ----")
print(aug_r["checker_feedback"].output_str)

print("---- new input ----")
new_input = patch_code_with_extra(
    aug_r["module"].code,
    aug_r["pos_to_preds"],
    aug_r["checker_feedback"].error_dict["env_code_2.py"],
)
print(new_input)


---- model output ----
<pad><s><extra_id_0>int<extra_id_1>int<extra_id_2>int<extra_id_3>int<extra_id_4>int, y : int<extra_id_5>int<extra_id_6>Optional[int]<extra_id_7>int<extra_id_8>int<extra_id_9>Bar[int, int, int, float, float]</s>
---- checker_feedback ----
env_code_2.py:20:14: error: Incompatible types in assignment (expression has type "str", variable has type "int")  [assignment]
env_code_2.py:32:29: error: Argument 1 to "len" has incompatible type "int"; expected "Sized"  [arg-type]
env_code_2.py:35:6: error: "Bar" expects no type arguments, but 5 given  [type-arg]
Found 3 errors in 1 file (checked 1 source file)

---- new input ----
# Env example 2: some existing annotations

from typing import *


def fib(n: /* int */<extra_id_0>):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


def foo(bar: /* int */<extra_id_1>):
    return fib(bar)


class Bar:
    z: /* int */<extra_id_2> = /* error: Incompatible types in

In [34]:
def run_model_with_extra(code: str, num_beams=16):
    input_ids = tokenizer.encode(code, return_tensors="pt").to(device)
    dec = model.generate(
        input_ids,
        max_length=max_target_length,
        num_beams=num_beams,
    )[0]
    return {
        "predicted_types": output_ids_as_types(dec, tokenizer),
        "generation": tokenizer.decode(dec),
    }


run_model_with_extra(new_input)


{'predicted_types': [int,
  int,
  int,
  int,
  int,
  int,
  None,
  int,
  int,
  Bar[int, int, int, float, float]],
 'generation': '<pad><s><extra_id_0>int<extra_id_1>int<extra_id_2>int<extra_id_3>int<extra_id_4>int<extra_id_5>int<extra_id_6>None<extra_id_7>int<extra_id_8>int<extra_id_9>Bar[int, int, int, float, float]</s>'}

In [42]:
from spot.utils import insert_strings

print(insert_strings(aug_r["module"].code, [(19, 13, "/*[assignment]*/")]))


# Env example 2: some existing annotations

from typing import *


def fib(n: int):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


def foo(bar: int):
    return fib(bar)


class Bar:
    z: int = /*[assignment]*/"hello"
    w: int

    def __init__(self, x: Any):
        self.x: int = x
        self.y: Optional[int] = None
        self.reset(self.z)

    def reset(self, w0):
        self.w = w0

    def foo(self, z: int) -> int:
        return self.x + len(z)


bar: Bar[int, int, int, float, float] = Bar(3)




In [5]:
# Step 1: Replace all types to predict with special tokens
print(tokenizer.decode(result["input_ids"]))


<s>
@dataclass
class GitRepo:
    author:<extra_id_0>
    name:<extra_id_1>
    url:<extra_id_2>
    stars:<extra_id_3>
    forks:<extra_id_4>

    def authorname(self):
        return self.author + "__" + self.name

    def repo_dir(self, repos_dir:<extra_id_5>) -><extra_id_6>:
        return repos_dir / "downloaded" / self.authorname()

    def download(self, repos_dir:<extra_id_7>, timeout=None) -><extra_id_8>:
        pass
</s>


In [28]:
# Step 2: Tokenize using Byte Pair Encoding (BPE)
print(tokenizer.convert_ids_to_tokens(result["input_ids"]))


['<s>', 'Ċ', '@', 'data', 'class', 'Ċ', 'class', 'ĠGit', 'Repo', ':', 'Ċ', 'ĠĠĠ', 'Ġauthor', ':', '<extra_id_0>', 'Ċ', 'ĠĠĠ', 'Ġname', ':', '<extra_id_1>', 'Ċ', 'ĠĠĠ', 'Ġurl', ':', '<extra_id_2>', 'Ċ', 'ĠĠĠ', 'Ġstars', ':', '<extra_id_3>', 'Ċ', 'ĠĠĠ', 'Ġfor', 'ks', ':', '<extra_id_4>', 'Ċ', 'Ċ', 'ĠĠĠ', 'Ġdef', 'Ġauthor', 'name', '(', 'self', '):', 'Ċ', 'ĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġself', '.', 'author', 'Ġ+', 'Ġ"__', '"', 'Ġ+', 'Ġself', '.', 'name', 'Ċ', 'Ċ', 'ĠĠĠ', 'Ġdef', 'Ġrepo', '_', 'dir', '(', 'self', ',', 'Ġrepos', '_', 'dir', ':', '<extra_id_5>', ')', 'Ġ->', '<extra_id_6>', ':', 'Ċ', 'ĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġrepos', '_', 'dir', 'Ġ/', 'Ġ"', 'down', 'loaded', '"', 'Ġ/', 'Ġself', '.', 'author', 'name', '()', 'Ċ', 'Ċ', 'ĠĠĠ', 'Ġdef', 'Ġdownload', '(', 'self', ',', 'Ġrepos', '_', 'dir', ':', '<extra_id_7>', ',', 'Ġtimeout', '=', 'None', ')', 'Ġ->', '<extra_id_8>', ':', 'Ċ', 'ĠĠĠĠĠĠĠ', 'Ġpass', 'Ċ', '</s>']


In [29]:
# Step 3: Let model predict a sequence of types using BPE
print(tokenizer.convert_ids_to_tokens(result["output_ids"]))


['<pad>', '<s>', '<extra_id_0>', 'str', '<extra_id_1>', 'str', '<extra_id_2>', 'str', '<extra_id_3>', 'List', '[', 'str', ']', 'Ġ+', 'ĠList', '[', 'str', ']', '<extra_id_4>', 'List', '[', 'str', ']', 'Ġ+', 'ĠList', '[', 'str', ']', 'Ġ+', 'ĠList', '[', 'str', ']', '<extra_id_5>', 'Path', '<extra_id_6>', 'Path', 'Ġ.', 'ĠPath', '<extra_id_7>', 'Path', 'Ġ.', 'ĠPath', '<extra_id_8>', 'Path', 'Ġ.', 'ĠPath', 'Ġ[', 'Ġstr', ']', '</s>']


In [30]:
# Step 4: Extract the predicted types
print(result["predicted_types"])


[str, str, str, Any, Any, Path, Path.Path, Path.Path, Path.Path[str]]
